In [1]:
import json
import pandas as pd

In [2]:
def read_json_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)
    return json_data

In [3]:
def json_to_dataframe(json_data):
    df = pd.DataFrame(json_data)
    return df

In [4]:
def extract_info(title):
        brand = "BMW"
        model = None
        car_type = None
        
        # Define the possible models
        possible_models = [
            "1M", "3GT", "5GT", "6GT", "i3", "i4", "i5", "i7", "i8", "Inny", "iX", "iX1", "iX2", "iX3",
            "M2", "M3", "M4", "M5", "M6", "M8", "Seria 1", "Seria 2", "Seria 3", "Seria 4", "Seria 5",
            "Seria 6", "Seria 7", "Seria 8", "X1 M", "X2 M", "X3 M", "X4 M", "X5 M", "X6 M", "X7 M", "XM", "Z1 M", "Z3 M", "Z4 M", "Z8 M",
            "X1", "X2", "X3", "X4", "X5", "X6", "X7", "Z1", "Z3", "Z4", "Z8"
        ]
        
        # Try to match the model from the title
        for m in possible_models:
            if m in title:
                model = m
                _car_type = title.split(m)
                if len(_car_type) > 1 and _car_type[1]:
                    car_type = _car_type[1].strip()
                break
        
        return brand, model, car_type

In [5]:
def test_extract_info():
    # Prepare sample data
    data = [
        "BMW M5",
        "BMW X1 xDrive18d xLine",
        "BMW M3",
        "BMW Seria 5",
        "BMW X5 3.0 d Edition Exclusive",
        "BMW Seria 3 330i Sport Line",
        "BMW X6 40d xDrive",
        "BMW Seria 1 116i",
        "BMW X5 xDrive25d sport",
        "BMW Seria 3 320d DPF Touring Edition Fleet",
    ]
    # Define expected results
    expected_brands = ["BMW"] * len(data)
    expected_models = ["M5", "X1", "M3", "Seria 5", "X5", "Seria 3", "X6", "Seria 1", "X5", "Seria 3"]
    expected_car_types = [
        None,
        "xDrive18d xLine",
        None,
        None,
        "3.0 d Edition Exclusive",
        "330i Sport Line",
        "40d xDrive",
        "116i",
        "xDrive25d sport",
        "320d DPF Touring Edition Fleet"
    ]
    
    for t,b,m,c in zip(data, expected_brands, expected_models, expected_car_types):
        print(f"Title {t}")
        got_b, got_m, got_c = extract_info(t)
        assert got_b == b, f"Expected brands {b}, but got {got_b}"
        assert got_m == m, f"Expected models {m}, but got {got_m}"
        assert got_c == c, f"Expected car types {c}, but got {got_c}"
        print(f"Title {t} passed")

# Run the test
test_extract_info()

Title BMW M5
Title BMW M5 passed
Title BMW X1 xDrive18d xLine
Title BMW X1 xDrive18d xLine passed
Title BMW M3
Title BMW M3 passed
Title BMW Seria 5
Title BMW Seria 5 passed
Title BMW X5 3.0 d Edition Exclusive
Title BMW X5 3.0 d Edition Exclusive passed
Title BMW Seria 3 330i Sport Line
Title BMW Seria 3 330i Sport Line passed
Title BMW X6 40d xDrive
Title BMW X6 40d xDrive passed
Title BMW Seria 1 116i
Title BMW Seria 1 116i passed
Title BMW X5 xDrive25d sport
Title BMW X5 xDrive25d sport passed
Title BMW Seria 3 320d DPF Touring Edition Fleet
Title BMW Seria 3 320d DPF Touring Edition Fleet passed


In [6]:
def convert_price(row):
    if row['price_currency'] == "EUR":
        return row['price'] * 4.33
    return row['price']

In [7]:
#read data
file_path = '../../data.json'
json_data = read_json_from_file(file_path)
df = json_to_dataframe(json_data)
#print(df.info())
#print(df.head(5))

In [8]:
cleaned_data = df.drop_duplicates(subset="id")
cleaned_data = cleaned_data.drop(columns=['url', 'image_url'])
cleaned_data[['brand', 'model', 'car_type']] = cleaned_data['title'].apply(lambda x: pd.Series(extract_info(x)))
cleaned_data = cleaned_data.drop(columns=['title', 'brand'])
cleaned_data.columns = cleaned_data.columns.str.lower().str.strip().str.replace(' ', '_')
cleaned_data['price'] = cleaned_data['price'].str.replace(' ', '').astype(int)
cleaned_data['year'] = cleaned_data['year'].str.replace(' ', '').astype(int)
cleaned_data['price'] = cleaned_data.apply(convert_price, axis=1)
cleaned_data = cleaned_data.drop(columns=['price_currency'])

cleaned_data.to_json('../../cleaned_base_data.json', orient='records', lines=True)

print(cleaned_data.info())
print(cleaned_data.head(5))

<class 'pandas.core.frame.DataFrame'>
Index: 6377 entries, 0 to 6399
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              6377 non-null   object 
 1   img_local       5274 non-null   object 
 2   type_of_seller  4206 non-null   object 
 3   price           6377 non-null   float64
 4   mileage         6377 non-null   object 
 5   fuel_type       6377 non-null   object 
 6   gearbox         6377 non-null   object 
 7   year            6377 non-null   int64  
 8   model           6377 non-null   object 
 9   car_type        4723 non-null   object 
dtypes: float64(1), int64(1), object(8)
memory usage: 806.1+ KB
None
                id            img_local       type_of_seller     price  \
0  619888553496191  619888553496191.jpg                 None  255000.0   
1  878738710972359  878738710972359.jpg                 None  249600.0   
2  325534139055352  325534139055352.jpg  Prywatny sprzedawca   5299